In [7]:
import os

%pwd

'd:\\Desktop\\MLOPS\\DataScienceProject'

In [2]:
os.chdir("../")
%pwd

'd:\\Desktop\\MLOPS\\DataScienceProject'

In [8]:
from dataclasses import dataclass
from pathlib import Path

# only difference in data class and normal class is in normal class we need self keyword to 
# assign value here we dont need it 

@dataclass()
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path
    

In [9]:
# Reload the constants module to get updated paths
import importlib
import src.datascienceproject.constants
importlib.reload(src.datascienceproject.constants)
from src.datascienceproject.constants import *
from src.datascienceproject.utils.common import read_yaml,create_directories

In [10]:
# Verify the paths are correct
print(f"CONFIG_FILE_PATH: {CONFIG_FILE_PATH}")
print(f"CONFIG_FILE_PATH exists: {CONFIG_FILE_PATH.exists()}")
print(f"PARAMS_FILE_PATH: {PARAMS_FILE_PATH}")
print(f"PARAMS_FILE_PATH exists: {PARAMS_FILE_PATH.exists()}")
print(f"SCHEMA_FILE_PATH: {SCHEMA_FILE_PATH}")
print(f"SCHEMA_FILE_PATH exists: {SCHEMA_FILE_PATH.exists()}")


CONFIG_FILE_PATH: config\config.yaml
CONFIG_FILE_PATH exists: True
PARAMS_FILE_PATH: params.yaml
PARAMS_FILE_PATH exists: True
SCHEMA_FILE_PATH: schema.yaml
SCHEMA_FILE_PATH exists: True


In [11]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,
                params_filepath=PARAMS_FILE_PATH,
                schema_filepath=SCHEMA_FILE_PATH):

        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config
        

In [12]:
from src.datascienceproject import logger
from urllib import request
import os
import zipfile

In [13]:
# data ingestion component

class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config
    
    # downloading zip file from github link in config.yaml file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers=request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f"file: {filename} downloaded with following info: {headers}")
        else:
            logger.info(f"file already exists of size")
    
    # extract zipfile
    def extract_zip_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)
    

    

   
        
            

In [15]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2025-10-14 23:24:11,164]:INFO:common:yaml file: config\config.yaml loaded successfully
[2025-10-14 23:24:11,167]:INFO:common:yaml file: params.yaml loaded successfully
[2025-10-14 23:24:11,169]:INFO:common:yaml file: schema.yaml loaded successfully
[2025-10-14 23:24:11,171]:INFO:common:created directory at: artifacts
[2025-10-14 23:24:11,171]:INFO:common:created directory at: artifacts/data_ingestion
[2025-10-14 23:24:12,191]:INFO:3241061009:file: artifacts/data_ingestion/data.zip downloaded with following info: Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: D70A:3AF089:1AEBE4:393E7F:68EE8E41
Accept-Ranges: bytes
Date: Tue, 14 Oct 2025 1